In [10]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [11]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [12]:
df


,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


Step 1: Create source embeddings for the text column

In [13]:
from sentence_transformers import SentenceTransformer


In [14]:
# Converting text into a vector
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

C:\Users\sdama\anaconda3\envs\AP_project_solve\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
vectors.shape


(8, 768)

In [18]:
dim = vectors.shape[1]
dim

768

Step 2 : Build a FAISS Index for vectors

In [20]:
# Creating an index that lets me search faster

import faiss

index = faiss.IndexFlatL2(dim)

Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [21]:
index.add(vectors)


In [22]:
index


<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000024D1BF7FC90> >

Step 4 : Encode search text using same encorder and normalize the output vector


In [40]:
search_query = "looking for places to visit during the holidays"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [41]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

Step 5: Search for similar vector in the FAISS index created


In [42]:
distances, I = index.search(svec, k=2)
distances

array([[0.92737263, 1.1601744 ]], dtype=float32)

In [43]:
I


array([[6, 7]], dtype=int64)

In [46]:
# Grid search finds our "result" in the index 6 and 7, both related to vacations
df.loc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [47]:
search_query

'looking for places to visit during the holidays'